In [71]:
from googlesearch import search
import json
import requests
from bs4 import BeautifulSoup
import re

# load json data from operaBookmarksExtractor.ipynb
with open('Data/games.json', 'r') as f:
    games_dict = json.load(f)

psnp_urls = []
i = 0
# find all PSNProfiles trophy list urls for each game (there can be many urls as each game could be released more than once on different system)
for g in games_dict['title']:
	print("Getting URLs for", g)
	query = g + " PSNProfiles"
	searchResults = search(query, num_results=8, sleep_interval=5)
	urls = []
	for j in searchResults:
		if "psnprofiles.com/trophies/" in j and j.count('/') == 4:
			urls.append(j)
	psnp_urls.append(urls)
	i += 1
	if i == 5:
		break
games_dict["psnprofilesTrophyList"] = psnp_urls

with open('Data/games_with_urls.json', 'w', encoding='utf-8') as f:
    json.dump(games_dict, f)


Getting URLs for L.A. Noire


HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DL.A.%252BNoire%252BPSNProfiles%26num%3D10%26hl%3Den%26start%3D0&hl=en&q=EhAqAqMaokIIgDElIV9FhGz3GLbUqbEGIjDBU6DKtFFJvPV46lRa-Sq7pug3ODPqaySlpAYXexVOvOfhm0OAqIwxITVJJgvG2-EyAXJaAUM

In [49]:
import html

platinum_percents = []
for i in range(len(games_dict["title"])):
    platsForGame = []
    for u in games_dict["psnprofilesTrophyList"][i]:
        response = requests.get("http://webcache.googleusercontent.com/search?q=cache:" + u + "&vwsrc=1")
        htmlCode = html.unescape(response.text)
        soup = BeautifulSoup(htmlCode, 'html.parser')
        stats = soup.find('div', class_='stats flex')
        for s in stats.find_all('span', class_='stat grow'):
            if s.find('span').text == 'Platinum Achievers':
                proc = s.text
        match = re.search(r'\d\d\.\d\d', proc).group()
        platsForGame.append(float(match))
    platinum_percents.append(platsForGame)
    
games_dict['platinumPercents'] = platinum_percents

KeyError: 'psnprofilesTrophyList'

In [70]:
response = requests.get("http://webcache.googleusercontent.com/search?q=cache:https://psnprofiles.com/trophies/8956-borderlands&vwsrc=1")
htmlCode = html.unescape(response.text)
soup = BeautifulSoup(htmlCode, 'html.parser')
stats = soup.find('div', class_='stats flex')
for s in stats.find_all('span', class_='stat grow'):
    if s.find('span').text == 'Platinum Achievers':
        proc = s.text
# match = re.search(r'\(.*\)', proc).group()
match = re.search(r'\d\d\.\d\d', proc).group()
print(match)

29.08


In [ ]:
import html

with open("Data/temp2.html", 'w') as f:
    f.write(response.text)
# print(response.text)

r = html.unescape(response.text)
print(r)
with open("Data/temp2.html", 'w', encoding='utf-8') as f:
    f.write(r)